In [2]:

#!unzip all-dogs
!pip install tqdm
#Import everything that is needed from Keras library.
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import glob
#matplotlib will help with displaying the results
import matplotlib.pyplot as plt
#numpy for some mathematical operations
import numpy as np
#PIL for opening,resizing and saving images
from PIL import Image
#tqdm for a progress bar when loading the dataset
from tqdm import tqdm
import scipy as sp
#os library is needed for extracting filenames from the dataset folder.
import os

def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = np.resize(image, new_shape)
        # store
        images_list.append(new_image)
    return np.asarray(images_list)
def get_fid(act1, act2):
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sp.linalg.sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


class FaceGenerator:
    #RGB-images: 3-channels, grayscale: 1-channel, RGBA-images: 4-channels
    def __init__(self,image_width,image_height,channels):
        self.image_width = image_width
        self.image_height = image_height

        self.channels = channels

        self.image_shape = (self.image_width,self.image_height,self.channels)

        #Amount of randomly generated numbers for the first layer of the generator.
        self.random_noise_dimension = 100

        #Just 10 times higher learning rate would result in generator loss being stuck at 0.
        optimizer = Adam(0.0002,0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
        self.generator = self.build_generator()

        #A placeholder for the generator input.
        random_input = Input(shape=(self.random_noise_dimension,))

        #Generator generates images from random noise.
        generated_image = self.generator(random_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        #Discriminator attempts to determine if image is real or generated
        validity = self.discriminator(generated_image)

        #Combined model = generator and discriminator combined.
        #1. Takes random noise as an input.
        #2. Generates an image.
        #3. Attempts to determine if image is real or generated.
        self.combined = Model(random_input,validity)
        self.combined.compile(loss="binary_crossentropy",optimizer=optimizer)

    def get_training_data(self,datafolder):
        print("Loading training data...")

        training_data = []
        #Finds all files in datafolder

        image_list = []
        for filename in glob.glob('./all-dogs/*.jpg'): #assuming gif
              
              im=Image.open(filename)
              image = Image.open(filename).convert('RGB')
              #Resizes to a desired size.
              image = image.resize((self.image_width,self.image_height),Image.ANTIALIAS)
              #Creates an array of pixel values from the image.
              pixel_array = np.asarray(image)

              training_data.append(pixel_array)
       
        
        
        #training_data is converted to a numpy array
        training_data = np.reshape(training_data,(-1,self.image_width,self.image_height,self.channels))
        return training_data


    def build_generator(self):
        #Generator attempts to fool discriminator by generating new images.
        model = Sequential()

        model.add(Dense(256*4*4,activation="relu",input_dim=self.random_noise_dimension))
        model.add(Reshape((4,4,256)))

        #Four layers of upsampling, convolution, batch normalization and activation.
        # 1. Upsampling: Input data is repeated. Default is (2,2). In that case a 4x4x256 array becomes an 8x8x256 array.
        # 2. Convolution: If you are not familiar, you should watch this video: https://www.youtube.com/watch?v=FTr3n7uBIuE
        # 3. Normalization normalizes outputs from convolution.
        # 4. Relu activation:  f(x) = max(0,x). If x < 0, then f(x) = 0.


        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))


        # Last convolutional layer outputs as many featuremaps as channels in the final image.
        model.add(Conv2D(self.channels,kernel_size=3,padding="same"))
        # tanh maps everything to a range between -1 and 1.
        model.add(Activation("tanh"))

        # show the summary of the model architecture
        model.summary()

        # Placeholder for the random noise input
        input = Input(shape=(self.random_noise_dimension,))
        #Model output
        generated_image = model(input)

        #Change the model type from Sequential to Model (functional API) More at: https://keras.io/models/model/.
        return Model(input,generated_image)


    def build_discriminator(self):
        #Discriminator attempts to classify real and generated images
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.image_shape, padding="same"))
        #Leaky relu is similar to usual relu. If x < 0 then f(x) = x * alpha, otherwise f(x) = x.
        model.add(LeakyReLU(alpha=0.2))

        #Dropout blocks some connections randomly. This help the model to generalize better.
        #0.25 means that every connection has a 25% chance of being blocked.
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        #Zero padding adds additional rows and columns to the image. Those rows and columns are made of zeros.
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        #Flatten layer flattens the output of the previous layer to a single dimension.
        model.add(Flatten())
        #Outputs a value between 0 and 1 that predicts whether image is real or generated. 0 = generated, 1 = real.
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        input_image = Input(shape=self.image_shape)

        #Model output given an image.
        validity = model(input_image)

        return Model(input_image, validity)

    def train(self, datafolder ,epochs,batch_size,save_images_interval):
        #Get the real images
        training_data = self.get_training_data(datafolder)

        #Map all values to a range between -1 and 1.
        training_data = training_data / 127.5 - 1.

        #Two arrays of labels. Labels for real images: [1,1,1 ... 1,1,1], labels for generated images: [0,0,0 ... 0,0,0]
        labels_for_real_images = np.ones((batch_size,1))
        labels_for_generated_images = np.zeros((batch_size,1))
        mfid=[]
        for epoch in range(epochs):
            # Select a random half of images
            
            indices = np.random.randint(0,len(training_data),batch_size)
            real_images = training_data[indices]

            #Generate random noise for a whole batch.
            random_noise = np.random.normal(0,1,(batch_size,self.random_noise_dimension))
            #Generate a batch of new images.
            generated_images = self.generator.predict(random_noise)

            #Train the discriminator on real images.
            discriminator_loss_real = self.discriminator.train_on_batch(real_images,labels_for_real_images)
            #Train the discriminator on generated images.
            discriminator_loss_generated = self.discriminator.train_on_batch(generated_images,labels_for_generated_images)
            #Calculate the average discriminator loss.
            discriminator_loss = 0.5 * np.add(discriminator_loss_real,discriminator_loss_generated)

            #Train the generator using the combined model. Generator tries to trick discriminator into mistaking generated images as real.
            generator_loss = self.combined.train_on_batch(random_noise,labels_for_real_images)
            print ("%d [Discriminator loss: %f, acc.: %.2f%%] [Generator loss: %f]" % (epoch, discriminator_loss[0], 100*discriminator_loss[1], generator_loss))
            images1 = scale_images(real_images, (16,16,3))
            images2 = scale_images(generated_images, (16,16,3))
            act1 = images1.reshape(batch_size,768  )
            act2 = images2.reshape(batch_size,768  )
            temp=(get_fid(act1, act2))
            print('fid : ' + str(temp))
    
            mfid.append(temp)

            if epoch % save_images_interval == 0:
                self.save_images(epoch)

        #Save the model for a later use
        #self.generator.save("saved_models/facegenerator.h5")
        for i in range(0,5000):
            print(mfid[i])


    def save_images(self,epoch):
        #Save 25 generated images for demonstration purposes using matplotlib.pyplot.
        rows, columns = 5, 5
        noise = np.random.normal(0, 1, (rows * columns, self.random_noise_dimension))
        generated_images = self.generator.predict(noise)

        generated_images = 0.5 * generated_images + 0.5

        figure, axis = plt.subplots(rows, columns)
        image_count = 0
        for row in range(rows):
            for column in range(columns):
                axis[row,column].imshow(generated_images[image_count, :], cmap='spring')
                axis[row,column].axis('off')
                image_count += 1
        figure.savefig('./Imagenet/dogs/generated_' + str(epoch)+ '.png')
        #figure.show()
        plt.close()

    def generate_single_image(self,model_path,image_save_path):
        noise = np.random.normal(0,1,(1,self.random_noise_dimension))
        model = load_model(model_path)
        generated_image = model.predict(noise)
        #Normalized (-1 to 1) pixel values to the real (0 to 256) pixel values.
        generated_image = (generated_image+1)*127.5
        print(generated_image)
        #Drop the batch dimension. From (1,w,h,c) to (w,h,c)
        generated_image = np.reshape(generated_image,self.image_shape)

        image = Image.fromarray(generated_image,"RGB")
        #image.save(image_save_path)

if __name__ == '__main__':
    facegenerator = FaceGenerator(64,64,3)
    facegenerator.train(datafolder="test",epochs=5000, batch_size=32, save_images_interval=100)
    #facegenerator.generate_single_image("saved_models/facegenerator.h5","test.png")
    

You should consider upgrading via the 'pip install --upgrade pip' command.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_11 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_12 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 17, 17, 64)        0         
_________________________________________________________________
batch_normalization_9 (Batch (None, 17, 17, 64)        256       
_________________________________________________________________
l

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 [Discriminator loss: 2.444091, acc.: 37.50%] [Generator loss: 0.148781]
fid : 254.74416763731804


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

fid : 116.90726264384551
4803 [Discriminator loss: 0.535237, acc.: 71.88%] [Generator loss: 0.689912]
fid : 178.1318573204588
4804 [Discriminator loss: 0.879540, acc.: 37.50%] [Generator loss: 0.626376]
fid : 108.12912136931217
4805 [Discriminator loss: 0.799266, acc.: 43.75%] [Generator loss: 0.744563]
fid : 104.8629067915802
4806 [Discriminator loss: 0.884443, acc.: 35.94%] [Generator loss: 0.676619]
fid : 85.17711968397073
4807 [Discriminator loss: 0.729627, acc.: 59.38%] [Generator loss: 0.748536]
fid : 82.75372465632223
4808 [Discriminator loss: 0.715851, acc.: 45.31%] [Generator loss: 0.847667]
fid : 103.37042962378992
4809 [Discriminator loss: 0.751607, acc.: 39.06%] [Generator loss: 0.849778]
fid : 92.86646217123769
4810 [Discriminator loss: 0.673745, acc.: 57.81%] [Generator loss: 1.049848]
fid : 71.68027848828173
4811 [Discriminator loss: 0.768724, acc.: 43.75%] [Generator loss: 0.911096]
fid : 73.06376304192223
4812 [Discriminator loss: 0.711404, acc.: 57.81%] [Generator los

4884 [Discriminator loss: 0.648580, acc.: 67.19%] [Generator loss: 0.671054]
fid : 48.950096620641034
4885 [Discriminator loss: 0.693680, acc.: 60.94%] [Generator loss: 0.613971]
fid : 78.72261465994262
4886 [Discriminator loss: 0.944285, acc.: 26.56%] [Generator loss: 0.656273]
fid : 58.38365359971997
4887 [Discriminator loss: 0.632132, acc.: 70.31%] [Generator loss: 0.656456]
fid : 65.40368077973223
4888 [Discriminator loss: 0.733871, acc.: 50.00%] [Generator loss: 0.896257]
fid : 96.01081101547082
4889 [Discriminator loss: 0.732420, acc.: 56.25%] [Generator loss: 0.743140]
fid : 71.71582318273116
4890 [Discriminator loss: 0.663366, acc.: 64.06%] [Generator loss: 0.738137]
fid : 71.06052182137343
4891 [Discriminator loss: 0.690845, acc.: 53.12%] [Generator loss: 0.808102]
fid : 65.26159187943811
4892 [Discriminator loss: 0.873287, acc.: 37.50%] [Generator loss: 0.796925]
fid : 65.61068406473896
4893 [Discriminator loss: 0.821618, acc.: 42.19%] [Generator loss: 0.672185]
fid : 60.9896

4965 [Discriminator loss: 0.867923, acc.: 34.38%] [Generator loss: 0.726428]
fid : 84.85940782498719
4966 [Discriminator loss: 0.785259, acc.: 68.75%] [Generator loss: 0.784570]
fid : 71.38023303246831
4967 [Discriminator loss: 0.597698, acc.: 65.62%] [Generator loss: 0.768779]
fid : 92.03965394220872
4968 [Discriminator loss: 0.920765, acc.: 35.94%] [Generator loss: 0.526209]
fid : 102.88154321081642
4969 [Discriminator loss: 0.862651, acc.: 45.31%] [Generator loss: 0.616104]
fid : 78.2121570824759
4970 [Discriminator loss: 0.753326, acc.: 42.19%] [Generator loss: 0.615848]
fid : 115.73079107573535
4971 [Discriminator loss: 0.763795, acc.: 45.31%] [Generator loss: 0.689263]
fid : 194.55910410598233
4972 [Discriminator loss: 0.627320, acc.: 67.19%] [Generator loss: 0.654253]
fid : 75.49344458343158
4973 [Discriminator loss: 0.655075, acc.: 65.62%] [Generator loss: 0.758613]
fid : 121.77113794202413
4974 [Discriminator loss: 0.679904, acc.: 59.38%] [Generator loss: 0.730050]
fid : 94.81

88.47102859723445
85.76976841023507
71.2645348959105
79.6725228973978
50.3440561594569
81.06617515949739
70.97243118446085
73.84528225979327
69.65214041035588
93.52057874292896
90.80179423790536
68.45267955507913
98.30126233830791
94.57938240180627
104.79430800468509
64.59714622619093
88.36553048062464
70.00599407866879
77.26174856515345
82.75675907338461
71.96896060037305
79.41708186329596
97.83546864608277
66.37132671306044
89.39737415583201
79.30543344606356
76.59278605351074
103.83210095628542
66.25145705598052
89.76552806452104
110.62846622784635
54.86880381704567
70.53307906713454
61.23308431432677
68.65822734539051
86.83173674138428
101.08469392581655
124.84520040995595
82.66529936650112
82.44516331787071
146.39042087534062
250.35083389193898
99.81503186738661
119.40815920439478
190.98462847773135
100.70612157233404
76.5713227929272
98.55454830146772
99.29053355424523
95.08020701613816
94.61299591695087
138.22460539636657
69.90499634589035
102.05173087857457
148.23025957779336
1

79.04012594921183
79.6873754867808
91.65064752475723
60.55332147450138
72.93481279517479
69.03799130910144
80.0103589409818
66.93221158018592
77.53356101863662
112.12386258199822
85.76392636672293
72.36367671962446
116.56929349591712
70.24648854515142
77.82169442638738
74.2434686891001
83.41501700463182
103.33251599736435
90.71220173598829
113.31632742177368
95.99189490459787
110.44524892810826
83.32787699795324
84.7197518221098
159.0885572214866
107.68861780554955
96.38437185673595
94.12500938883045
94.51084753901756
81.32728008420875
108.93508722727647
119.17697761485158
211.38263079460663
108.4944456667395
81.660156334028
105.31803831514803
132.06901263063526
96.13974617119152
163.53109027472482
65.98781427454654
73.84513916952325
61.16384441546799
84.47450336132007
81.2753111569041
76.65481428926876
73.57543512904022
84.73991698777664
84.86265526687876
80.04263567798816
65.8476960305425
79.00195564768057
76.09751423238727
88.40536495241031
76.34513163161009
103.17996300011166
81.14